In [4]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import Ollama
llm_model = Ollama(model = "llama2-uncensored")

In [5]:
response = llm_model.invoke("Who is rohit sharma and what team does he play for in ipl 2025")
print("Ollama says:", response)

Ollama says: Rohit Sharma is an Indian cricketer who plays as a right-handed opening batsman and occasional right-arm off break bowler. He currently represents the Mumbai Indians team in the Indian Premier League (IPL).


In [7]:
from langchain_community.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [9]:
loader = WebBaseLoader("https://en.wikipedia.org/wiki/Amazon_(company)")

In [12]:
def WebsiteLoader(urls):
    loader = WebBaseLoader(urls)
    return loader.load()

In [13]:
urls = ["https://en.wikipedia.org/wiki/Amazon_(company)", "https://en.wikipedia.org/wiki/Apple_Inc."]
website_docs = WebsiteLoader(urls)


In [16]:
#splitting documents into chunks
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 100,
    length_function = len,
    is_separator_regex = False,
)

all_documents = website_docs

split_documents = text_splitter.split_documents(all_documents)

In [22]:
# Embedding Documents and storing data in chroma

#initiliazing Ollama embeddings
from langchain_ollama import OllamaEmbeddings
embeddings = OllamaEmbeddings(
    model="llama2-uncensored",
)
persist_directory = "./chroma_db"

#store documents in chromadb
vectorstore = Chroma.from_documents(
    documents = split_documents,
    embedding = embeddings,
    persist_directory = persist_directory
)

vectorstore.persist()
print("data succesfully stored in chromadb")

#Reload the vector store for retrieval
vectorstore = Chroma(persist_directory=persist_directory, embedding_function=embeddings)
print("Chromadb reloaded successfully")

data succesfully stored in chromadb
Chromadb reloaded successfully


/var/folders/mt/c6qz9wtd1813dmvdwcp8l1780000gn/T/ipykernel_1839/925210441.py:17: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()
/var/folders/mt/c6qz9wtd1813dmvdwcp8l1780000gn/T/ipykernel_1839/925210441.py:21: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(persist_directory=persist_directory, embedding_function=embeddings)


In [3]:
#retrieving most relevant documents using similarity search

from langchain.chains import RetrievalQA
retriever = vectorstore.as_retriever(search_type = "similarity", search_kwargs={"k":5})
qa = RetrievalQA.from_chain_type(
    llm = llm, chain_type = "stuff", retriever=retriever, return_source_documents=True)
query = "query?"
result = qa({"query":query})
print(result['result'])


IndentationError: unexpected indent (1967958172.py, line 7)